# Vectorizing the studentized bootstrap

In this post we'll examine three non-parametric tests: 1) the [Mann-Whitney U](https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test) (MNU) test, 2) [Wilcoxon signed rank](https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test) (WSR) test, and 3) the [Spearman correlation coefficient](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient). I've chosen these specific tests for three reasons.

In [45]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import plotnine
from plotnine import *
import seaborn as sns
import matplotlib.pyplot as plt
from time import time

"""
SIMPLE HELPER FUNCTIONS
"""
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def rvec(x):
    return np.atleast_2d(x)

def cvec(x):
    return rvec(x).T

def to_3d(mat):
    return np.atleast_3d(mat).transpose(2,0,1)

## (Studentized) bootstrap support functions

Suppose we have either one or two vector(s) of data $x$ and $y$. We may want to know something about the distribution of $x$, or the relationship between $x$ and $y$. It is also possible that $x$ and $y$ might be "paired", meaning they are jointly measured from some unit of observation (e.g. height and weight from the same person). The WSR and Spearman statistics are example of statistics for "paired" data, and hence require the lengths of the two vectors to be equivalent. In contrast, the MNU can test for differences between any two distributions, of possibly different lengths. 

The code block below provides for...


In [2]:
"""
BOOTSTRAPER: CARRIES OUT STUDENTIZED BOOTSTRAP
             ALLOWS FOR PAIRED SAMPLES
"""
def bs_gen(x, n_bs, n_s, y=None, is_paired=True):
    n_x = len(x)
    assert not ((y is None) and (is_paired==True))
    if y is not None:
        n_y = len(y)
    else:
        y_bs, y_s = None, None
    # (i) Carry out the bootstrap
    if is_paired:  # Implies y exists
        assert len(x) == len(y)
        x_bs = pd.Series(x).sample(frac=n_bs,replace=True)
        x_bs_idx = x_bs.index
        x_bs = x_bs.values.reshape([n_x,n_bs])
        y_bs = pd.Series(y).iloc[x_bs_idx]
        y_bs = y_bs.values.reshape([n_y,n_bs])
    else:
        x_bs = pd.Series(x).sample(frac=n_bs,replace=True).values.reshape([n_x,n_bs])
        if y is not None:
            y_bs = pd.Series(x).sample(frac=n_bs,replace=True).values.reshape([n_y,n_bs])
    # (ii) Studentize
    x_s = pd.DataFrame(x_bs).sample(frac=n_s,replace=True)
    x_s_idx = x_s.index
    x_s = x_s.values.reshape([n_s, n_x, n_bs]).transpose(1,2,0)
    if is_paired:
        y_s = pd.DataFrame(y_bs).iloc[x_s_idx]
        y_s = y_s.values.reshape([n_s, n_y, n_bs]).transpose(1,2,0)
    return x_bs, x_s, y_bs, y_s

"""
FUNCTION GENERATE CIs
"""

def gen_CI(stat, stat_bs, se_bs, se_s, pvals):
    tt = ['student','normal','quant']
    z_q = np.quantile(stat_bs,pvals.flat).reshape(pvals.shape)
    z_n = stats.norm.ppf(pvals)
    t_s = (stat_bs - stat) / se_s
    z_s = np.quantile(t_s,pvals.flat).reshape(pvals.shape)
    df = pd.DataFrame(np.r_[stat - se_bs*z_s[:,[1,0]], 
                            stat - se_bs*z_n[:,[1,0]],
                            z_q],columns=['lb','ub'])
    df.insert(0,'stat',stat)
    df = df.assign(tt=np.repeat(tt,len(pvals)),alpha=np.tile(2*pvals[:,0],len(tt)))
    return df

## (1) Spearman correlation coefficient

The Spearman correlation coefficient will undergo the following transformation in order to ensure it is [consistent](https://www.tse-fr.eu/sites/default/files/medias/stories/SEMIN_09_10/STATISTIQUE/croux.pdf).


In [3]:
"""
WRAPPER TO CALCULAE SPEARMAN'S CORRELATION COEFFICIENT WITH ADJUSTMENT FOR CONSISTENCY
"""
def spearman_trans(x):
    return 2*np.sin(np.pi * x / 6)

def rho_spearman(x,y):
    return spearman_trans(stats.spearmanr(x,y)[0])

"""
VECTORIZED PAIRWISE SPEARMAN CORRELATION
A : (n x p x s)
n: sample size (rows of the data)
p: columns of the data (could be bootstrapped columns)
s: copies of the n x p matrix (could be studentized copies)
"""
def pairwise_cor(A, B):
    assert A.shape == B.shape
    n = A.shape[0]
    if (len(A.shape) == 2):
        mu_A, mu_B = rvec(A.mean(0)), rvec(B.mean(0))
        se_A, se_B = A.std(axis=0,ddof=1), B.std(axis=0,ddof=1)
    else:
        mu_A, mu_B = to_3d(A.mean(0)), to_3d(B.mean(axis=0))
        se_A, se_B = A.std(axis=0,ddof=1), B.std(axis=0,ddof=1)
    D = np.sum((A - mu_A) * (B - mu_B),0) / (n-1)
    return D / (se_A*se_B)

def pairwise_spearman(A, B):
    return spearman_trans(pairwise_cor(A, B))

"""
CARRY OUT BOOTSTRAP FOR SPEARMAN
"""
def bs_student_spearman(x, y, n_bs, n_s, alpha=0.05):
    if isinstance(alpha, float) | isinstance(alpha,list):
        alpha = np.array([alpha])
    alpha = rvec(alpha)
    assert len(x) == len(y)
    assert np.all(alpha > 0) & np.all(alpha < 0.5)
    # (i) Get baseline statistic
    rho = rho_spearman(x, y)
    n = len(x)
    pvals = np.r_[alpha/2,1-alpha/2].T
    # (ii) Transform data into ranks and sample with replacement
    x_r, y_r = stats.rankdata(x), stats.rankdata(y)
    # (iii) Get the bootstrapped and studentized samples
    x_bs, x_s, y_bs, y_s = bs_gen(x=x_r, y=y_r, n_bs=n_bs, n_s=n_s, is_paired=True)
    rho_bs = pairwise_spearman(x_bs, y_bs)
    se_rho_bs = rho_bs.std(ddof=1)
    se_rho_s = pairwise_spearman(x_s, y_s).std(axis=1,ddof=1)
    # (iv) Get the confidence intervals for the different approaches
    df = gen_CI(stat=rho, stat_bs=rho_bs, se_bs=se_rho_bs, se_s=se_rho_s, pvals=pvals)
    return df

Simulations below will do two things. First, we'll compare the type-1 error rate when the data comes from the same distribution. Then we'll examine the power, for different sample sizes. We'll use exponential data to show how the statistic functions for non-Gaussian data.

In [48]:
seed = 1234
nsim = 1000
n_gt = int(1e6)
n_seq = list(np.arange(25,101,25))
scale_seq = [0, 5, 10]
n_bs, n_s = 1000, 1000
np.random.seed(seed)

stime = time()
holder = []
for k, scale in enumerate(scale_seq):
    v1_gt = np.random.exponential(size=n_gt)
    v2_gt = np.random.exponential(size=n_gt)
    if scale > 0:
        v2_gt = v1_gt + np.random.exponential(scale=scale,size=n_gt)
    rho_gt = rho_spearman(v1_gt, v2_gt)
    print('rho_gt: %0.3f' % rho_gt)
    for j, n in enumerate(n_seq):
        print('n: %i' % n)
        for i in range(nsim):
            v1 = np.random.exponential(size=n)
            v2 = np.random.exponential(size=n)
            if scale > 0:
                v2 = v1 + np.random.exponential(scale=scale,size=n)
            tmp_df = bs_student_spearman(v1, v2, n_bs=n_bs, n_s=n_s, alpha=[0.05, 0.10, 0.2])
            tmp_df = tmp_df.assign(rho=rho_gt,lam=scale,n=n, sim=i)
            holder.append(tmp_df)
            if (i + 1) % 100 == 0:        
                nleft = nsim*(len(n_seq)-(j+1))*(len(scale_seq)-(k+1)) + (nsim - (i+1))
                nsec = time() - stime
                rate = (i+1) / nsec
                eta = nleft / rate
                print('ETA: %i seconds (%i of %i)' % (eta, i+1, nsim))
df_sim_spearman = pd.concat(holder).reset_index(None, True)
df_sim_spearman = df_sim_spearman.assign(rho=lambda x: np.where(x.lam==0,0,x.rho))
df_sim_spearman.to_csv(os.path.join('../output/df_sim_spearman.csv'),index=False)

# cn_gg = ['tt','n','alpha','scale']
# dat_cov = df_sim.groupby(cn_gg).apply(lambda x: np.mean((x.lb<=0) & (x.ub>=0))).reset_index()
# dat_cov.rename(columns={0:'coverage'},inplace=True)
# dat_cov.pivot_table('coverage',cn_gg[:-1],'alpha')
# df_sim.groupby(['alpha','tt']).apply(lambda x: pd.Series({'err_lb':np.mean(x.lb > 0),
#                                                           'err_ub':np.mean(x.ub < 0),
#                                                           'spread':np.mean(x.ub - x.lb)}))

rho_gt: 0.001
n: 25
ETA: 3667 seconds (100 of 1000)
ETA: 3603 seconds (200 of 1000)
ETA: 3552 seconds (300 of 1000)
ETA: 3495 seconds (400 of 1000)
ETA: 3475 seconds (500 of 1000)
ETA: 3439 seconds (600 of 1000)
ETA: 3379 seconds (700 of 1000)
ETA: 3318 seconds (800 of 1000)
ETA: 3263 seconds (900 of 1000)
ETA: 3207 seconds (1000 of 1000)
n: 50
ETA: 31294 seconds (100 of 1000)
ETA: 17912 seconds (200 of 1000)
ETA: 13350 seconds (300 of 1000)
ETA: 11002 seconds (400 of 1000)
ETA: 9533 seconds (500 of 1000)
ETA: 8535 seconds (600 of 1000)
ETA: 7778 seconds (700 of 1000)
ETA: 7173 seconds (800 of 1000)
ETA: 6680 seconds (900 of 1000)
ETA: 6265 seconds (1000 of 1000)
n: 75
ETA: 50550 seconds (100 of 1000)
ETA: 27008 seconds (200 of 1000)
ETA: 18973 seconds (300 of 1000)
ETA: 14906 seconds (400 of 1000)
ETA: 12344 seconds (500 of 1000)
ETA: 10749 seconds (600 of 1000)
ETA: 9410 seconds (700 of 1000)
ETA: 8358 seconds (800 of 1000)
ETA: 7499 seconds (900 of 1000)
ETA: 6775 seconds (1000 of 1

In [49]:
!ls -lt ../output | head

total 3473144
-rwxrwxrwx 1 edrysdale edrysdale   10308578 Feb 11 16:20 df_sim_spearman.csv
-rwxrwxrwx 1 edrysdale edrysdale       6400 Feb 10 11:39 dat_suspect.csv
-rwxrwxrwx 1 edrysdale edrysdale       1195 Feb 10 11:38 dup_test.csv
-rwxrwxrwx 1 edrysdale edrysdale   49599805 Feb  2 17:18 y_agg.csv
-rwxrwxrwx 1 edrysdale edrysdale         93 Feb  2 12:40 best_outcome.csv
-rwxrwxrwx 1 edrysdale edrysdale        124 Feb  2 12:39 best_mdl.csv
-rwxrwxrwx 1 edrysdale edrysdale      54587 Feb  2 12:31 cpt_anno_organ.csv
-rwxrwxrwx 1 edrysdale edrysdale      83222 Feb  2 12:31 cpt_anno_group.csv
-rwxrwxrwx 1 edrysdale edrysdale      44374 Feb  2 12:31 cpt_anno.csv


## (2) Mann-Whitney U statistic

1. DISCUSS FORMULA FOR STATISTIC, AS WELL AS ASYMPTOTIC DIST
2. MENTION THAT WHEN THERE ARE "TIES", THE FORMULA FOR THE SE NEEDS TO SHRINK
3. THIS MEANS THAT THE BOOTSTRAP VARIANCE WILL BE "CONSERVATIVE" LINK THE PREVIOUS POST, AND WILL BE UNDERPOWERED
4. THIS MEANS THAT UNFORTUNATELY WE STILL NEED TO RANK THE BOOTSTRAPS. HOWEVER, WE CAN BOOTSTRAP THE RANKS FOR THE STUDENTIZED COMPONENT

In [ ]:
"""
BOOTSTRAP MWU
"""
def bs_student_mwu(x, y, n_bs, n_s, alpha=0.05):
    if isinstance(alpha, float) | isinstance(alpha,list):
        alpha = np.array([alpha])
    alpha = rvec(alpha)
    assert np.all(alpha > 0) & np.all(alpha < 0.5)
    # (i) Get baseline statistic
    pvals = np.r_[alpha/2,1-alpha/2].T    
    n_x, n_y = len(x), len(y)
    n_xy = n_x * n_y
    # (ii) Transform to ranks and bootstrap
    z = stats.rankdata(np.append(x, y))[:n_x]
    z_bs, z_s, _, _ = bs_gen(x=z, n_bs=n_bs, n_s=n_s, is_paired=False)
    # Calculate the MWU
    mu = n_xy/2
    se = np.sqrt((n_xy*(n_x+n_y+1))/12)
    mnu = ((z.sum(0) - n_x*(n_x+1)/2) - mu) / se
    mnu_bs = ((z_bs.sum(0) - n_x*(n_x+1)/2) - mu) / se
    mnu_s = ((z_s.sum(0) - n_x*(n_x+1)/2) - mu) / se
    se_mnu_bs = mnu_bs.std(ddof=1)
    se_rho_s = mnu_s.std(axis=1,ddof=1)
    df = gen_CI(stat=mnu, stat_bs=mnu_bs, se_bs=se_mnu_bs, se_s=se_rho_s, pvals=pvals)
    return df

In [ ]:
nsim = 1000
n_bs = 900
n1_seq = [50, 100, 150]
n2_seq = n1_seq.copy()

holder = []
for n1 in n1_seq:
    for n2 in n2_seq:
        x1x2 = np.random.randn(n1+n2, nsim)
        z = stats.rankdata(x1x2,axis=0)
        z1, z2 = z[:n1], z[n1:]
        mu1, mu2 = n1*(n1+1)/2 + n1*n2/2, n2*(n2+1)/2 + n1*n2/2
        mnu1 = z1.sum(0) - mu1
        mnu2 = z2.sum(0) - mu2
        holder.append(pd.DataFrame({'u1':mnu1, 'u2':mnu2, 'n1':n1, 'n2':n2}))
df_distU = pd.concat(holder).reset_index(None,True)
df_distU = df_distU.melt(['n1','n2'],None,'tt','u')
dat_se = df_distU.groupby(['n1','n2','tt']).u.std(ddof=1).reset_index()
dat_se = dat_se.assign(theory=lambda x: np.sqrt((x.n1*x.n2*(x.n1 + x.n2 + 1))/12))
dat_se.tail()

In [ ]:
# Calculate the standard error of the bootstrapped ranks
se_bs = np.mean([np.std(pd.Series(z1[:,j]).sample(frac=n_bs,replace=True).values.reshape([n1,n_bs]).sum(0) - mu1,ddof=1) for j in range(nsim)])
print(se_bs)

In [ ]:
z1_bs = pd.DataFrame(z1).sample(frac=n_bs,replace=True).values.reshape([n_bs, n1, nsim]).transpose(1,2,0)
mnu1_bs = z1_bs.sum(0) - mu1
print(z1.sum(0).std())

In [ ]:
z1_bs = pd.Series(z1[:,0]).sample(frac=n_bs,replace=True).values.reshape([n1,n_bs])
mask_z1_bs = pd.DataFrame(z1_bs).apply(lambda x: x.duplicated(), 0)

In [ ]:
df = pd.DataFrame({'a':[1,2,1,1,2,3],'b':list(range(6,0,-1))})
# Sort the unique values in each
df = df.apply(lambda x: x.sort_values().values, 0)
# Find the duplicates
mask = df.apply(lambda x: x.duplicated(), 0)

In [ ]:
mask

In [ ]:
# The mean ranks of the bootstrap line up with the ranks of that realization
sns.scatterplot(z1_bs.mean(0).mean(1), z1.mean(0))

In [ ]:
sns.scatterplot(z1_bs.std(0).mean(1), z1.std(0))

In [ ]:
se_bs2 = np.mean(
    [np.std(stats.rankdata(pd.Series(x1x2[:,j]).sample(frac=n_bs,replace=True).values.reshape([n1+n2,n_bs]),axis=0)[:n1].sum(0) - mu1) for j in range(nsim)])
print(se_bs2)

In [ ]:
# Calculate the standard error for ranking bootstraps

In [ ]:
plotnine.options.figure_size = (8,7)
gg_distU = (ggplot(df_distU,aes(x='u',fill='tt')) + theme_bw() + 
            geom_density(alpha=0.5,color='black') + 
            facet_wrap('~n1+n2',scales='free',labeller=label_both) + 
            theme(subplots_adjust={'wspace': 0.05,'hspace':0.45},
                  axis_text_y=element_blank()))
gg_distU